In [2]:
import json
import os
import pickle
import sys

import numpy as np
from tqdm.notebook import tqdm

with open('../dirs.json', 'r') as f:
    dirs = json.load(f)
sys.path.append(dirs['root'])

from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import StandardScaler

from utils.ipm import NeuronLoader
from utils.metrics import DescriminationConfidenceEstimator as DCE
from utils.utils import mkeqdata
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import kendalltau
from matplotlib import pyplot as plt
import seaborn as sns

from scipy.ndimage import gaussian_filter1d as smooth

nmb_rep = 100

outPath = os.path.join(dirs['out']['rsa'])
os.makedirs(outPath, exist_ok=True)

In [3]:
from typing import Union

def hist_equalize(matrix):
    matrix = np.copy(matrix)
    shape = matrix.shape
    matrix = matrix.reshape(-1)
    index = matrix.argsort()
    values = np.linspace(0, 1, index.shape[0])
    matrix[index] = values
    return matrix.reshape(shape)

def mkrdm(data):
    rdmat = 1 - cosine_similarity(data)
    return rdmat

def mkherdm(data):
    rdmat = 1 - cosine_similarity(data)
    return hist_equalize(rdmat)

def movavg(inp, bl, ss):
    nbin = int(np.floor((inp.shape[2] - bl) / ss))
    out = np.nan * np.zeros((inp.shape[0], inp.shape[1], nbin))
    for ibin in range(nbin):
        out[:, :, ibin] = np.sum(inp[:, :, int(ibin*ss):int(ibin*ss)+bl], 2) / bl * 1000
    return out

In [49]:
monkey = "both"
selectivity = "fast"

# grangerInfluence = {}
grangerInfluence = {'itc2pfc': {}, 'pfc2itc': {}}
with open("../utils/info.pkl", "rb") as handler:
    info = pickle.load(handler)
info = info[:165]
sIndex = ((info.sfr == "A") | (info.sfr == "BI"))
noneIndex = (info.cat == "none")
info = info[~noneIndex & sIndex].reset_index(drop=True)
itc = np.load(f"G:/Data/{selectivity.capitalize()}/{monkey.capitalize()}/ITCNoAvg.npy")
pfc = np.load(f"G:/Data/{selectivity.capitalize()}/{monkey.capitalize()}/PFCNoAvg.npy")
itc = itc[~noneIndex & sIndex]
pfc = pfc[~noneIndex & sIndex]

bl, ss, n, k = 50, 1, 50, 25

itcb = movavg(itc, bl, ss)
pfcb = movavg(pfc, bl, ss)
timb = np.arange(-200+bl/2, 700-bl/2, ss)

# argsort = np.argsort(info.cat)
argsort = np.argsort(
    info.con.astype('category').cat.set_categories(
        ['human face', 'monkey face', 'human body', 'monkey body', 'artificial', 'natural'], 
        ordered=True
    )
)
rdmItc = np.nan * np.ones([argsort.size, argsort.size, timb.size])
rdmPfc = np.nan * np.ones([argsort.size, argsort.size, timb.size])

for itime in range(timb.size):
    rdmItc[:,:,itime] = mkherdm(itcb[argsort, :, itime])
    rdmPfc[:,:,itime] = mkherdm(pfcb[argsort, :, itime])

In [10]:
with open("../utils/info.pkl", "rb") as handler:
    info = pickle.load(handler)
info = info[:165]
argsort = np.argsort(
    info.con.astype('category').cat.set_categories(
        ['human face', 'monkey face', 'human body', 'monkey body', 'artificial', 'natural'], 
        ordered=True
    )
)

info.iloc[argsort]

,img,sfr,con,sup,mid,fac,bod,art,nat,cat
0,<PIL.Image.Image image mode=L size=50x50 at 0x...,A,human face,animate,face,True,False,False,False,face
164,<PIL.Image.Image image mode=L size=50x50 at 0x...,C,human face,animate,face,True,False,False,False,face
57,<PIL.Image.Image image mode=L size=50x50 at 0x...,A,artificial,inanimate,none,False,False,True,False,artificial
164,<PIL.Image.Image image mode=L size=50x50 at 0x...,C,human face,animate,face,True,False,False,False,face
58,<PIL.Image.Image image mode=L size=50x50 at 0x...,A,artificial,inanimate,none,False,False,True,False,artificial
...,...,...,...,...,...,...,...,...,...,...
90,<PIL.Image.Image image mode=L size=50x50 at 0x...,BH,natural,inanimate,none,False,False,False,True,natural
91,<PIL.Image.Image image mode=L size=50x50 at 0x...,BH,natural,inanimate,none,False,False,False,True,natural
92,<PIL.Image.Image image mode=L size=50x50 at 0x...,BH,natural,inanimate,none,False,False,False,True,natural
94,<PIL.Image.Image image mode=L size=50x50 at 0x...,BH,natural,inanimate,none,False,False,False,True,natural


In [48]:
argsort = np.argsort(
    info.con.astype('category').cat.set_categories(
        ['human face', 'monkey face', 'human body', 'monkey body', 'artificial', 'natural'], 
        ordered=True
    )
)


0     human face
1     human face
2     human face
3     human face
4     human face
         ...    
68    artificial
69    artificial
70    artificial
71    artificial
72    artificial
Name: con, Length: 73, dtype: category
Categories (6, object): ['human face' < 'monkey face' < 'human body' < 'monkey body' < 'artificial' < 'natural']

In [52]:
cm = 1/2.54

for i, t in tqdm(enumerate(timb), total=timb.size):
    fig, axs = plt.subplots(1,1,figsize=(5*cm, 5*cm), dpi=200)
    ax = axs
    ax.imshow(rdmItc[:,:,i], cmap='viridis')
    ax.axis('off')
    plt.savefig(f"./rdm/itc({t:.0f}).pdf", dpi=400)
    plt.close()

    fig, axs = plt.subplots(1,1,figsize=(5*cm, 5*cm), dpi=200)
    ax = axs
    ax.imshow(rdmPfc[:,:,i], cmap='viridis')
    ax.axis('off')
    plt.savefig(f"./rdm/pfc({t:.0f}).pdf", dpi=400)
    plt.close()

  0%|          | 0/850 [00:00<?, ?it/s]